#### Deploy a Machine Learning model on Google Cloud Run
***
1. Build the App and ML model
2. Setup Google Cloud
3. Install and initialize Google Cloud SDK 
4. Prepare Dockerfile, requirements.txt, .dockerignore
5. Cloud build & Deploy

#### 1. Building a Machine Learning model (neural network) and an app (using flask)
- Solve a classification problem to detect hand written digits (MNIST dataset)
- Run the ML model in an app
- Send a request to the app for prediction

In [1]:
# To avoid displaying more than 2 messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from tensorflow import keras
from flask import Flask, request, jsonify

#importing the dataset
mnist = keras.datasets.mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)

# Normalize: 0,255 -> 0,1
x_train, x_test = x_train / 255.0, x_test / 255.0

(60000, 28, 28) (60000,)


In [4]:
model = keras.models.Sequential([
                                keras.layers.Flatten(input_shape=(28,28)),
                                keras.layers.Dense(128, activation='relu'),
                                keras.layers.Dense(10)
                                ])

# loss and optimizer
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ['accuracy']

model.compile(loss=loss, optimizer=optim, metrics=metrics)


In [7]:
#training
batch_size = 64
epochs = 5

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2)

model.save('nn.h5')

# Evaluate the saved model:
print('Evaluating the saved model:')
print('current model: ',model.evaluate(x_test, y_test, batch_size=batch_size))
new_model = keras.models.load_model('./nn.h5')
print('saved model: ',new_model.evaluate(x_test, y_test, batch_size=batch_size, verbose=2))

Epoch 1/5
938/938 - 1s - loss: 0.0173 - accuracy: 0.9951 - 1s/epoch - 2ms/step
Epoch 2/5
938/938 - 1s - loss: 0.0152 - accuracy: 0.9959 - 1s/epoch - 2ms/step
Epoch 3/5
938/938 - 1s - loss: 0.0120 - accuracy: 0.9970 - 1s/epoch - 2ms/step
Epoch 4/5
938/938 - 1s - loss: 0.0111 - accuracy: 0.9969 - 1s/epoch - 2ms/step
Epoch 5/5
938/938 - 1s - loss: 0.0089 - accuracy: 0.9979 - 1s/epoch - 2ms/step
Evaluating the saved model:
157/157 [==============================] - 0s 1ms/step - loss: 0.0856 - accuracy: 0.9766
current model:  [0.08555935323238373, 0.9765999913215637]
157/157 - 0s - loss: 0.0856 - accuracy: 0.9766 - 262ms/epoch - 2ms/step
saved model:  [0.08555935323238373, 0.9765999913215637]


In [15]:
def transform_image(pillow_image):
    data = np.asarray(pillow_image)
    data = data / 255.0
    data = data[np.newaxis, ..., np.newaxis]
    # --> [1, x, y, 1]
    data = tf.image.resize(data, [28, 28])
    return data


def predict(x):
    predictions = model(x)
    predictions = tf.nn.softmax(predictions)
    pred0 = predictions[0]
    label0 = np.argmax(pred0)
    return label0

In [ ]:
app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        file = request.files.get('file')
        if file is None or file.filename == "":
            return jsonify({"error": "no file"})

        try:
            image_bytes = file.read()
            pillow_img = Image.open(io.BytesIO(image_bytes)).convert('L')
            tensor = transform_image(pillow_img)
            prediction = predict(tensor)
            data = {"prediction": int(prediction)}
            return jsonify(data)
        except Exception as e:
            return jsonify({"error": str(e)})

    return "OK"

app.run(debug=True)

#### 2. Setup Google Cloud Platform
- Create an account
- Create a new project
- Set up Google Cloud Run & Google Cloud Build APIs

#### 3. Install Google Cloud SDK
- Install Google Cloud SDK
- Set up our Project in Google Cloud SDK

#### 4. Create Dockerfile, requirements.txt, .dockerignore
- We need a dockerfile for all the commands/instructions to run on the cloud
- requirements.txt lists all the required libraries to setup on the cloud
- .dockerignore excludes defined files/directories to be uploaded in the image for docker build process

#### 5. Cloud build & deploy the app/ML model